第4回 モデルの構築

今回はいよいよモデルの構築に入っていきます。

モデルとはNN(ニューラルネットワーク)全般のことを指します。NNには種類がいくつかあり、有名どころではCNN（放送局?）やRNN、発展としてGANやLSTMなどがありますが、これらも一般にはモデルと呼ばれます。

Pytorchのモデルの構築には、torchの中のnn.Moduleクラスを利用することがほとんどです。以下のコードはそのモデル構築の一例を示しています。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#nn.Moduleクラスを親にして、モデルを定義する
class NeuralNetwork(nn.Module):
    def __init__(self):
        #super().__init__()は、親クラスのメソッドやフィールドの使用に不可欠
        super().__init__()
        #二次元のデータを一次元にする
        self.flatten = nn.Flatten()
        #ここで、重みを含んだ層を定義する
        self.linear_relu_stack = nn.Sequential(
            #28*28=784の入力→512の出力（この入出力の数を間違えると動作しない）
            nn.Linear(28*28, 512),
            nn.ReLU(),
            #512の入力→512の出力
            nn.Linear(512, 512),
            nn.ReLU(),
            #512の入力→10の最終出力
            nn.Linear(512, 10),
        )

    #注目3
    def forward(self, x):
        #二次元→一次元
        x = self.flatten(x)
        #入力データxを入れた時の最終出力をlogitsに代入
        logits = self.linear_relu_stack(x)
        return logits

これで、モデルの定義は完了です。このモデルのクラスのインスタンスを制作すれば、モデルの実装は完了します。

In [2]:
model=NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


それでは、このモデルにランダムな二次元のデータを入力してみましょう。

実際にモデルに値を入力する場合は、モデルをまるで関数かのように扱って構いません。以下のコードを見ればわかります。

In [3]:
X = torch.rand(1, 28, 28)
#modelのforwardの戻り値が代入される（関数みたい）
logits = model(X)
print(logits)
#ソフトマックス関数を用いて、出力を0から1までの値に直す
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
#何番目の要素が最大か示す
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0190, -0.0991, -0.0045,  0.1534, -0.0401,  0.0489, -0.0955, -0.1130,
          0.0308,  0.0116]], grad_fn=<AddmmBackward0>)
tensor([[0.0991, 0.0914, 0.1005, 0.1177, 0.0970, 0.1060, 0.0918, 0.0902, 0.1041,
         0.1021]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([3])


1、レイヤーについて

モデルは複数の層（レイヤー）で構成されます。ここでは、さっきのモデルで使用された層について解説します。

フラット層...データの次元を一次元にする層。Pytorchでは、torch.nn.Flatten()で示される。

In [6]:
input_image = torch.rand(1,28,28)
print(input_image.size())
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
flatten2=torch.flatten(input_image)
views=input_image.view(-1,784)
print(views.size())
print(flatten2.size())

torch.Size([1, 28, 28])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([784])


このように、28*28の二次元のデータを一次元配列に変換できていますね。しかし、以下の場合はどうでしょう。

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])


おいおい、[3,784]の二次元のデータになってんじゃねーか?!って思うかもしれません。しかし、問題はありません。この３って数は「チャネル数」とも呼ばれ、フラット化だけでは一次元に直せない次元のデータです。

このチャンネル数を見かける場面とすれば、jpg画像データの学習の時くらいです。jpg画像データのチャネル数は３、pngの場合はチャネル数は4、グレースケールの画像の場合はチャネル数は１になります。

このチャネル数は、データの構成要素の数などを示します。jpg画像はRGBやGBRの3要素、pngはRGBAの4要素、グレースケールは黒(0)から白(1)までの間の数値1要素で構成されるので、それに対応したチャネル数となります。

しかし、そもそもなぜフラット化をする必要があるのでしょうか？答えは線形層の入力にあります。

線形層...nn.Linearで定義される層。重み、バイアスを含む一次元配列の層

In [7]:
#これではエラーが出ません
print(flat_image.size())
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 784])
torch.Size([3, 20])


この線形層に二次元のデータを入力してみましょう。

In [8]:
#しかし、これはエラーが出ます
error_tensor=torch.rand(3,28,28)
hidden2=nn.Linear(28*28,20)(error_tensor)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (84x28 and 784x20)

エラー文を見てみると、shape can't be multiplied と書いてあり、形が合っていないとの指摘がされていることがわかります。つまりは、Linear層の入力は28*28の一次元配列なのに、二次元配列のデータが入力されてしまいpythonがブチギレたということです。(ただし、チャネル数の次元は除く)

チャネル数を除く次元が1次元でないと、線形層は入力を受け付けません。だからこそ、フラット化が必要なわけですね。

また、入力のサイズが合っていないのも、エラーの原因になります。

In [9]:
#flat_imageは28*28の一次元配列なのに、入力は28*27→エラー
err_layer1 = nn.Linear(in_features=28*27, out_features=20)
hidden1 = err_layer1(flat_image)
print(hidden1.size())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x784 and 756x20)

ReLU層...活性化関数の一つReLU関数をもちいて、入力データを変化させる層

In [10]:
input_image = torch.rand(1,28,28)
flatten = nn.Flatten()
flat_image = flatten(input_image)
hidden1 = layer1(flat_image)
out_relu=nn.ReLU()(hidden1)
print(out_relu)

tensor([[0.1740, 0.5786, 0.0000, 0.1441, 0.0000, 0.0209, 0.0000, 0.0000, 0.6372,
         0.4300, 0.1162, 0.1729, 0.0000, 1.0431, 0.0000, 0.3307, 0.0000, 0.0141,
         0.1596, 0.0432]], grad_fn=<ReluBackward0>)


このReLU関数は以下のように示されます

In [11]:
def ReLU_model(x):
    if x>0:
        return x
    else:
        return 0

また、LinearとReLUとFlattenだけ使うと、モデルは以下のようにも定義し直すことができます。

In [12]:
class LazyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1=nn.Linear(28*28,512)
        self.Re1=nn.ReLU()
        self.fc2=nn.Linear(512,512)
        self.Re2=nn.ReLU()
        self.fc3=nn.Linear(512,10)
    def forward(self, x):
        #層を一つ一つ指定して処理を書く
        x = self.flatten(x)
        x=self.fc1(x)
        x=self.Re1(x)
        x=self.fc2(x)
        x=self.Re2(x)
        logits=self.fc3(x)
        return logits

しかし、これを見るとforwardのところが少し冗長（無駄に長い）ように感じます。

このforwardの処理は「順伝搬」とよばれます。順伝搬は、入力データをある層に入れてその層の出力を別の層に入力するという一連の処理のことを指します。この順伝搬の処理を書くにあたり、どの順番で層を通るのかを指定する必要があるので、このような冗長なコードとなってしまうのです。

そこで、いくつかの層をまとめて、一つの構造体を作ることで順伝搬の処理をより容易に書くことができます。それがnn.Sequentialです。

In [14]:
class SequentialModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        #ここで使う
        self.linear_seq=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits=self.linear_seq(x)
        return logits
    
input_data=torch.rand(1,28,28)
seq=SequentialModel()
logits=seq(input_data)
print(logits)
print(seq)

tensor([[-0.0277,  0.1467,  0.0084, -0.0111,  0.0598, -0.0301,  0.1294,  0.0153,
          0.0468,  0.0775]], grad_fn=<AddmmBackward0>)
SequentialModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_seq): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


このように順伝搬のところがすっきりします。

SoftMax層...ソフトマックス関数を用いて、モデルの最終出力を0から1までの値に直す層

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(logits)
print(pred_probab)

tensor([[-0.0277,  0.1467,  0.0084, -0.0111,  0.0598, -0.0301,  0.1294,  0.0153,
          0.0468,  0.0775]], grad_fn=<AddmmBackward0>)
tensor([[0.0932, 0.1109, 0.0966, 0.0947, 0.1017, 0.0929, 0.1090, 0.0972, 0.1004,
         0.1035]], grad_fn=<SoftmaxBackward0>)


補足，畳み込み層

ここからはCNNで使う層について説明します。CNNを学習するのはまだですが覚えておいて損はしません。

畳み込み層...データの特徴を抽出する層(?)。パラメータの削減にも繋がる便利な層。

この層が行う処理は、最初聞いただけでは「?」ですが、画像認識に限らず、色んな場面で使うので、理解していきましょう

In [16]:
input_conv=torch.rand(1,8,8)
#入力チャネル1、出力チャネル3、カーネルサイズ3*3の畳み込み層
conv1=nn.Conv2d(1,3,3)
hidden_conv=conv1(input_conv)
print(hidden_conv)
print(hidden_conv.size())

tensor([[[-0.0707, -0.3690, -0.2722, -0.2176, -0.6717, -0.4614],
         [-0.2187, -0.5794, -0.5742,  0.0082, -0.4647, -0.2522],
         [-0.4068, -0.2224, -0.4991, -0.4461, -0.4735, -0.4634],
         [-0.9353, -0.1874, -0.4183, -0.7560, -0.4998, -0.6876],
         [-0.2851, -0.5675,  0.1137, -0.4073, -0.6404, -0.4579],
         [-0.0427, -0.7599, -0.4247, -0.3312, -0.7007, -0.2728]],

        [[ 0.0102,  0.0813,  0.0586,  0.1369,  0.2562,  0.2612],
         [-0.0820,  0.1109,  0.0124, -0.0695,  0.2094,  0.0936],
         [-0.0428, -0.0328, -0.0650, -0.1414,  0.0691, -0.1170],
         [ 0.0359,  0.0802,  0.0599,  0.1983,  0.2090, -0.0931],
         [ 0.1813,  0.1071,  0.0158,  0.2180,  0.2374,  0.0338],
         [ 0.0835,  0.1605, -0.0829,  0.1908,  0.1380,  0.0526]],

        [[-0.3936,  0.0351,  0.2058, -0.1227,  0.2203,  0.3411],
         [ 0.1337, -0.1204, -0.2169,  0.0418, -0.2056,  0.0691],
         [ 0.0579, -0.2827, -0.0233, -0.2716, -0.1213, -0.0650],
         [-0.2874,  0

畳み込み層の処理はコードを実行しただけではさっぱりです。分かるとすれば、入力のデータが[1,8,8]だったのが[3,6,6]に変化したくらいです。

では、この畳み込み層の解説をしましょう。

畳み込み層では、カーネルというものがPytorchによって複数用意されます。カーネルとはランダムな数値を持つ小さな行列のことです。

このカーネルと画像のある一部分とのアダマール積を求めて、その成分の合計を一つの行列の値とします（語彙力）。

まあ見てもらった方が早いです<p><p><img src="https://camo.qiitausercontent.com/12545193254e3938b8914f22cdff6e238eab2d4b/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3232313834352f33636231326232312d383539612d393331312d326165362d3735363932383231313931302e676966">

上のgifだと、カーネルが以下の行列で示されます

In [ ]:
"""
|1 0 1|
|0 1 0|
|1 0 1|
"""

畳み込み層では、このカーネルの数が、出力のチャネル数に対応します。上のコードを例にすると、チャネル数１のデータがカーネル数3の畳み込み層に入力されると、出力チャネル数が3となります。

また、出力のサイズが変化しているのもこの畳み込みによる影響と言えます。そのため、情報の量が減少してしまうので、その分チャネル数を増やすことで情報を補っているというわけです。(諸説あり) 

この複数のカーネルというフィルターをかけて特徴を抽出するのが、畳み込み層の役割なんですね。

しかし、いくら畳み込んだとしても学習データはチャネル数１にしないといけません。そこで、viewというtensor型のメソッドを紹介します。

In [17]:
input_conv=torch.rand(1,8,8)
#入力チャネル1、出力チャネル3、カーネルサイズ3*3の畳み込み層
conv1=nn.Conv2d(1,3,3)
hidden_conv=conv1(input_conv)
print(hidden_conv.size())
#チャネル数3の6*6のデータなので、チャネル数1の要素3*6*6のデータに直す
view_hidden=hidden_conv.view(-1,3*6*6)
print(view_hidden.size())

torch.Size([3, 6, 6])
torch.Size([1, 108])


これにより線形層での学習ができるようになります．今回は以上です．お疲れ様でした．